In [1]:
!pip install pandas
!pip install scikit-learn
!pip install missingno
!pip install Jinja2
# !pip uninstall nbformat ipython plotly
!pip install nbformat ipython plotly
!pip install plotly
# !pip install nbformat --upgrade
!pip install geopandas
!pip install shapely

In [2]:
### import libraries
import pandas as pd 
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import missingno
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import datetime
from datetime import timedelta

In [12]:
# Data extraction
data = pd.read_csv('D:/Kaggle Datasets/playground-series-s3e20/train.csv')
# top 5 rows
data[:5]

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,emission
0,ID_-0.510_29.290_2019_00,-0.51,29.29,2019,0,-0.000108,0.603019,-0.000065,0.255668,-98.593887,...,3664.436218,61085.809570,2615.120483,15.568533,0.272292,-12.628986,35.632416,-138.786423,30.752140,3.750994
1,ID_-0.510_29.290_2019_01,-0.51,29.29,2019,1,0.000021,0.728214,0.000014,0.130988,16.592861,...,3651.190311,66969.478735,3174.572424,8.690601,0.256830,30.359375,39.557633,-145.183930,27.251779,4.025176
2,ID_-0.510_29.290_2019_02,-0.51,29.29,2019,2,0.000514,0.748199,0.000385,0.110018,72.795837,...,4216.986492,60068.894448,3516.282669,21.103410,0.251101,15.377883,30.401823,-142.519545,26.193296,4.231381
3,ID_-0.510_29.290_2019_03,-0.51,29.29,2019,3,NaN,NaN,NaN,NaN,NaN,...,5228.507736,51064.547339,4180.973322,15.386899,0.262043,-11.293399,24.380357,-132.665828,28.829155,4.305286
4,ID_-0.510_29.290_2019_04,-0.51,29.29,2019,4,-0.000079,0.676296,-0.000048,0.121164,4.121269,...,3980.598120,63751.125781,3355.710107,8.114694,0.235847,38.532263,37.392979,-141.509805,22.204612,4.347317


In [13]:
# data split
## Train-Valid
### Train-Test

trainData, validData = train_test_split(data,test_size=0.2,random_state=42)
trainData, testData = train_test_split(trainData,test_size=0.1,random_state=42)

print(f'Train Dataset:', trainData.shape)
print(f'Valid Dataset:',validData.shape)
print(f'Test Dataset:', testData.shape)

Train Dataset: (56896, 76)
Valid Dataset: (15805, 76)
Test Dataset: (6322, 76)


### Drop columns that has more than 90% missing values

In [14]:
columnsToDrop = {'ID_LAT_LON_YEAR_WEEK','UvAerosolLayerHeight_aerosol_height','UvAerosolLayerHeight_aerosol_pressure',
                          'UvAerosolLayerHeight_aerosol_optical_depth','UvAerosolLayerHeight_sensor_zenith_angle',
                          'UvAerosolLayerHeight_sensor_azimuth_angle','UvAerosolLayerHeight_solar_azimuth_angle',
                          'UvAerosolLayerHeight_solar_zenith_angle'}

In [15]:
# trainTarget = trainData['emission']
trainData.drop(columns = columnsToDrop,inplace=True, axis=1)
validData.drop(columns = columnsToDrop,inplace=True, axis=1)
testData.drop(columns = columnsToDrop,inplace=True, axis=1)

### Impute values with mean

In [16]:
from sklearn.impute import SimpleImputer

In [24]:
impute = SimpleImputer(strategy='mean')

trainImpute = impute.fit_transform(trainData)
validImpute = impute.transform(validData)
testImpute = impute.transform(testData)

In [29]:
trainImputeData = pd.DataFrame(trainImpute,columns=trainData.columns)
validImputeData = pd.DataFrame(validImpute,columns=validData.columns)
testImputeData = pd.DataFrame(testImpute,columns=testData.columns)

## Adding data points
1) Border country 
2) Nearby province
3) Industrial region

In [33]:
import geopandas as gpd
from shapely.geometry import Point

# Country data
# Path to the unzipped shapefile
rwanda_map = gpd.read_file('D:/Kaggle Datasets/RWA_adm/RWA_adm0.shp')
burundi_map = gpd.read_file('D:/Kaggle Datasets/Burundi/geoBoundaries-BDI-ADM0.shp')
drc_map = gpd.read_file('D:/Kaggle Datasets/DRC/cod_admbnda_adm0_rgc_itos_20190911.shp')
uganda_map = gpd.read_file('D:/Kaggle Datasets/Uganda/uga_admbnda_adm0_ubos_20200824.shp')
tanzania_map = gpd.read_file('D:/Kaggle Datasets/Tanzania/tza_admbnda_adm0_20181019.shp')
# Load the shapefile
rwanda_map = rwanda_map[['NAME_0','geometry']]
burundi_map = burundi_map[['shapeName','geometry']]
drc_map = drc_map[['ADM0_FR','geometry']]
uganda_map = uganda_map[['ADM0_EN','geometry']]
tanzania_map = tanzania_map[['ADM0_EN','geometry']]

mapData = pd.concat([rwanda_map,burundi_map,drc_map,uganda_map,tanzania_map],axis=0)
mapData['Country'] = np.where(mapData['NAME_0'].notnull(),mapData['NAME_0'],np.where(mapData['shapeName'].notnull(),mapData['shapeName'],
                                np.where(mapData['ADM0_FR'].notnull(),mapData['ADM0_FR'],mapData['ADM0_EN'])))
mapData = mapData[['Country','geometry']]

In [34]:
# Province data
rwanda_map = gpd.read_file('D:/Kaggle Datasets/RWA_adm/RWA_adm1.shp')

# Industrial region data
industialRegion = gpd.read_file('D:/Kaggle Datasets/RWA_adm/RWA_adm2.shp')

In [35]:
rwanda_map['Province_Name'] = np.where(rwanda_map['NAME_1']=='Amajyaruguru','Northern Province',
                                       np.where(rwanda_map['NAME_1']=='Amajyepfo', 'Southern Province',
                                                np.where(rwanda_map['NAME_1']=='Iburasirazuba','Eastern Province',
                                                         np.where(rwanda_map['NAME_1']=='Iburengerazuba','Western Province',
                                                                  np.where(rwanda_map['NAME_1']=='Umujyi wa Kigali','Kigali City','Rwanda')))))

In [36]:
industialRegion = industialRegion[(industialRegion['NAME_2'] == 'Gasabo')|(industialRegion['NAME_2'] == 'Kicukiro')|(industialRegion['NAME_2'] == 'Bugesera')|\
    (industialRegion['NAME_2'] == 'Huye')|(industialRegion['NAME_2'] == 'Rusizi')|(industialRegion['NAME_2'] == 'Nyabihu')|(industialRegion['NAME_2'] == 'Musanze')|\
    (industialRegion['NAME_2'] == 'Rwamagana')|(industialRegion['NAME_2'] == 'Rubavu')|(industialRegion['NAME_2'] == 'Muhanga')|\
    (industialRegion['NAME_2'] == 'Nyagatare')][['NAME_2','geometry']].reset_index(drop=True)

In [ ]:
class location:
    def __init__(self, latitude, longitude, targetMap, targetColumn):
        self.latitude = latitude
        self.longitude = longitude
        self.targetMap = targetMap
        self.targetColumn = targetColumn

    def find_location(self):
        point = Point(self.longitude, self.latitude)

        for idx, row in self.targetMap.iterrows():
            if row['geometry'].contains(point):
                return row[self.targetColumn]
            
        return "Unknown"
#### Need to get the nearest province and Industrial region
    